### This is the Gradio App that will interface with users trying to use our MediBot

In [ ]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

# Import Gradio
import gradio as gr

for each word in the conditions, find out how many times it appears in the input string, the one that appears the most is the prognosis, if ties, handle, if none, handle as prognosis unkown. put the prognosis medical terms into a list

### Building our MediBot model, this is where the model gets built and run from

In [ ]:
def prognosis_classification(medi_df):
    """
    Perform Prognosis classification using a pipeline with [TF-IDF vectorization and Linear Support Vector Classification] ### WE will have to decide whether this is the approach we should use.

    Parameters:
    - medi_df (pd.DataFrame): DataFrame containing 'input' and 'prognosis' columns for SMS classification.

    Returns:
    - text_clf (Pipeline): Fitted pipeline model for prognosis classification.

    This function takes a DataFrame with 'input' and 'prognosis' columns, splits the data into
    training and testing sets, builds a pipeline with [TF-IDF vectorization and Linear Support Vector
    Classification], and fits the model to the training data. 
    The fitted pipeline is returned to make future predictions.
    """
    # Set the features variable to the input column.
    x_features = medi_df['input']
    
    # Set the target variable to the prognosis column.
    y_target = medi_df['prognosis']
   
    # Split data into training and testing and set the test_size = 33%
    x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size=0.33, random_state=1)

    # Build a pipeline to transform the test set to compare to the training set.
    # how we build our model is what we should decide, this SVC is a placeholder for now
    medi_model = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english')), 
        ('clf', LinearSVC())
        ]
    )

    # Fit the model to the transformed training data and return model.
    medi_model.fit(x_train, y_train)
    return medi_model

### Load the dataset into the dataframe

In [ ]:
# Load the dataset into a DataFrame
medi_df = pd.read_csv("Resources/SMSSpamCollection.csv")
medi_df.head(5)

### Now we will build our model from our gathered data

In [ ]:
# Call the prognosis_classification function with the DataFrame and set the result to the "text_clf" variable
text_clf = prognosis_classification(medi_df)

### Building our function to predict and classify the prognosis from what the user inputted

In [ ]:
# The function should return the prognosis
def prog_prediction(text):
    """
    Predict the prognosis classification of a given text message of symptoms from the user using a pre-trained model.

    Parameters:
    - text (str): The text message of symptoms to be classified.

    Returns:
    - str: A message indicating the user's prognosis.

    This function takes a text message of symptoms and a pre-trained pipeline model, then predicts the
    prognosis of the user. The result is a message indicating what they might be afflicted with and how to solve it.
    """
    # Create a variable that will hold the prediction of a new set of symptoms.
    prediction = text_clf.predict([text])
    # Using a conditional if the prediction is "ham" return the message:
    # f'The text message: "{text}", is not spam.' Else, return f'The text message: "{text}", is spam.'
    return_msg = f"Based on your symptoms, we have deduced that your prognosis might be {prediction}"
    # in case the model just doesn't work
    return return_msg
    

### Build the Gradio MediBot App

In [ ]:
# Create a medi_bot_app that takes a textbox for the inputs and has a textbox for the output.  
# Povide labels for each textbox. 
app = gr.Interface(
    fn=sms_prediction,
    title='The Radical MediBot App (Please note that this is not medical advice, just a SUGGESTION, we are not doctors.)',
    inputs = [
        gr.Textbox(lines=2, placeholder="What seems to be troubling you today?", label="List out your symptoms here", interactive=True),
    ],
    outputs=gr.Textbox(lines=2, placeholder="Your prognosis is as follows: ", label="Our app has determined:", show_copy_button=True)
)
    
# Launch the app.
app.launch(show_error=True) 